## 텔레그램 챗봇 만들기
* 텔레그램은 카카오톡과 같은 유명한 인터넷 메신저
* bot은 특정작업을 수행하도록 고안된 자동화 프로그램
* 마치 인간이 하는 행동을 흉내내도록 만들었음
* 메신저 봇은 사람과 자동으로 대화하도록 고안된 프로그램
* 봇의 활용용도는 메세지를 보내거나 채팅방의 데이터를
* 수집하는 등의 다양한 용도로 사용가능
* core.telegram.org

### 파이썬용 텔레그램 라이브러리 설치
* python-teltgram-bot 이라는 패키지는
* 파이썬에서 텔레그램을 쉽게 이용할 수 있도록 지원하는 wrapper 패키지임
* pip install python-telegram-bot

In [1]:
# == 뒤에 버전을 지정해줌
!pip install python-telegram-bot==13.7

In [2]:
import telegram

In [3]:
# 실습에 사용하는 버전 : v13.15 (2022.05.06 버전)
# 2023.09.20 기준 v20.5가 최신버전
telegram.__version__

'13.7'

In [4]:
# 텔레그램 API 키 등록
with open('telegram_token.txt') as f:
    mytoken = f.read()
mytoken

'6540124535:AAFlmA7M3sEEaHtXK451bGTofe1dyEJ7miU'

### 텔레그램 메세지 보내기 1

In [5]:
# 텔레그램 봇 초기화
bot = telegram.Bot(token=mytoken)

In [6]:
# 대화상대에게 메세지 전송
bot.sendMessage(chat_id=5893411629, text='Hellow, World!!')

### 텔레그램 메세지 보내기 2

In [7]:
import requests

In [8]:
chat_id = 5893411629
text = 'hello, World!!'

url = f'https://api.telegram.org/bot{mytoken}/sendMessage'\
        f'?chat_id={chat_id}&text={text}'

In [9]:
# 메세지 보내고 결과 받음
requests.get(url).json()

{'ok': True,
 'result': {'message_id': 90,
  'from': {'id': 6540124535,
   'is_bot': True,
   'first_name': 'bigdata2199',
   'username': 'bigdata2199Bot'},
  'chat': {'id': 5893411629,
   'first_name': 'soomin',
   'last_name': 'choi',
   'type': 'private'},
  'date': 1695178180,
  'text': 'hello, World!!'}}

### 로또 텔레그램 봇
* /lotto라고 입력시 6개 숫자 출력

#### 파이썬으로 날짜시간 다루기
* datetime 패키지와 strftime함수를 이용해 날짜와 시간 출력 가능

In [10]:
import datetime as dt
import locale

#locale.setlocale(locale.LC_ALL, 'ko_KR.UTF-8')
dt.datetime.now()

datetime.datetime(2023, 9, 20, 11, 49, 41, 12034)

In [11]:
# 형식문자열 : %H(24시), %I(12시), %M(분), %S(초), %p(오전/오후)
now = dt.datetime.now()
now.strftime('%p %I:%M:%S')

'AM 11:49:41'

In [12]:
# 형식문자열 : %Y(년), %m(월), %d(일), %A(요일), %a(요일 축약)
now.strftime('%Y년 %m월 %d일 %A이고요 줄여서는 %a입니다.')

'2023년 09월 20일 Wednesday이고요 줄여서는 Wed입니다.'

### 텔레그램 봇 생성, 메세지 출력

In [13]:
import random
import time
from telegram import Update
from telegram.ext import Updater, CallbackContext
from telegram.ext import MessageHandler
from telegram.ext import CommandHandler
from telegram.ext import Filters

#### 텔레그램 챗봇 응답처리 콜백함수 정의
* 콜백함수명(context, update)

In [14]:
# 입력받은 메세지를 다시 메세지로 출력하는 함수
def reply_message(update, context):
    context.bot.send_message(text=update.message.text,
                    chat_id=update.message.chat_id)

In [15]:
# '/lotto'라 입력하면 로또를 출력하는 함수
def lotto_message(update, context):
    lotto = random.sample(range(1, 45+1), 6)
    update.message.reply_text(f'오늘의 행운의 숫자는 ? : {lotto} 🐈🐕🐅')

In [16]:
# '/date'라 입력하면 현재날짜(년월일요일)를 출력하는 함수
def date_message(update, context):
    date = now.strftime('%Y년 %m월 %d일 %A이고요 줄여서는 %a입니다.')
    update.message.reply_text(f'오늘은  {date} 🐥🐤🐣')

In [17]:
# '/time'라 입력하면 현재시간(시분초 오전/오후)을 출력하는 함수
def time_message(update, context):
    time = now.strftime('%p %I시 %M분 %S초입니다.')
    update.message.reply_text(f'현재시간은  {time} 🌝🌞⭐')

#### 텔레그램 챗봇 서버 실행
* Updater: 사용자로부터 새로운 메세지가 있는지 주기적으로 확인

In [18]:
print('텔레그램 RPA 서버 준비중...')
updater = Updater(mytoken, use_context=True)

텔레그램 RPA 서버 준비중...


### 콜백함수를 챗봇서버에 등록하기
* 위에서 정의한 콜백함수들을 핸들러객체로 만든후 챗봇서버에 등록
* MessageHandler : 입력받은 텍스트를 그대로 처리할때 사용할 콜백함수를 호출하는 핸들러
* CommandHandler : 특정문자열 입력시 이것과 관련된 콜백함수 호출CommandHandler(명령, 콜백함수)
* addHandler : 생성한 핸들러 객체를 챗봇서버에 등록함

#### 핸들러 정의

In [19]:
reply_handler = MessageHandler(Filters.text, reply_message)
lotto_handler = CommandHandler('lotto', lotto_message)
date_handler = CommandHandler('date', date_message)
time_handler = CommandHandler('time', time_message)

#### 핸들러 등록
* CommandHandler를 먼저 등록하고
* 맨 마지막에 MessageHandler를 등록함

In [20]:
updater.dispatcher.add_handler(lotto_handler)
updater.dispatcher.add_handler(date_handler)
updater.dispatcher.add_handler(time_handler)
updater.dispatcher.add_handler(reply_handler)

#### 챗봇서버 가동
* start_polling

In [21]:
print('텔레그램 RPA 서버 실행중...')
updater.start_polling(timeout=3, drop_pending_updates=True)
updater.idle()

텔레그램 RPA 서버 실행중...
